In [2]:
# pip install transformers datasets accelerate

# 抓dataset

## dataset裡分成三個 train(訓練用) validation(每次訓縣完測試訓練情形用) test(最終算分用)

In [3]:
import json
with open("train_social_att_sup.json",'r',encoding='utf-8')as f:
    train_data=json.load(f)
with open("dev_social_att_sup.json",'r',encoding='utf-8')as f:
    eval_data=json.load(f)

## 把每一筆資料集的每一項拆開

In [4]:
weight=[1/len(train_data) for x in range(len(train_data))]

In [5]:
def read_data(dataset):
    sentence1=[]
    sentence2=[]
    label=[]
    idx=[]
    for text in dataset:
        sentence1.append(text[0])
        sentence2.append(text[1])
        label.append(text[2])
        #idx.append(text['idx'])
    return sentence1,sentence2,label

In [6]:
ori_train_sen1,ori_train_sen2,ori_trainlabel=read_data(train_data)
print(len(ori_train_sen1))
print(ori_train_sen1[0])
eval_sen1,eval_sen2,evallabel=read_data(eval_data)
print(len(eval_sen1))
print(eval_sen1[0])
print(ori_trainlabel)

5334
Hotel部分似乎已經默默營業了 但是疫情影響 有多少人去住還未知  題外話 基泰這間公司的行銷真的很不行耶 管理部門不打算花點錢妝點一下新大樓嗎
730
怪怪~ 什麼狀況? 怎麼跳水跳這麼猛? 法說還沒開始啊~ 怎麼先來一場法會了?
[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 

In [7]:
print('train 資料集數量= ',len(train_data))
print('eval 資料集數量= ',len(eval_data))

train 資料集數量=  5334
eval 資料集數量=  730


# 選擇和使用tokenizer，具體要使用哪個tokenizer可以去huggingface官網找

In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [10]:
import transformers
transformers.logging.set_verbosity_error()

# 主要修改以下的東西

In [11]:
total_epoch=5
count_epoch=0
best_epoch = {"epoch": 0, "acc": 0 }
all_epoch=[]

In [12]:
from torch.utils import data
import torch
def add_targets(encodings,label):
    encodings.update({'label':label})

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [13]:
def eval_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

In [14]:
# import random
# #我不確定boosting會不會丟相同的測資進去，就是如果有一題實在錯太多了，那會重複訓練該筆測資嗎，會重複的話使用第二個程式
# def weighted_random_choice(elements, weights, nk):
#     selected_elements = []
#     total_weight = sum(weights)
#     while len(selected_elements) < nk:
#         choice = random.choices(elements, weights=weights, k=1)[0]
#         if choice not in selected_elements:
#             selected_elements.append(choice)
#     return selected_elements



# def weighted_random_choice2(elements, weights,k):
#     choice = random.choices(elements, weights=weights, k=k)
    # return choice

In [15]:
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)
add_targets(eval_encodings,evallabel)
eval_dataset = Dataset(eval_encodings)

In [17]:
import numpy as np
import math
import logging
from datasets import load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
from transformers import (
    AdamW,
    default_data_collator,
    get_scheduler,
    BertConfig,
    BertForSequenceClassification
)
from accelerate import Accelerator

alpha_list = [] #投票決定時需要alpha
all_epoch = []  # List to store epoch accuracy results
best_epoch = {'epoch': -1, 'acc': 0.0}  # Dictionary to store the best epoch
train_batch_size = 4      # 設定 training batch size 
eval_batch_size = 4      # 設定 eval batch size
num_train_epochs = 2    #設定單模型 epoch
data_collator = default_data_collator
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播
no_decay = ["bias", "LayerNorm.weight"]
output_dir = '.'

for count_epoch in range(total_epoch):
    print("epoch: ",count_epoch)
    train_sen1, train_sen2, trainlabel = read_data(train_data)
    train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
    add_targets(train_encodings, trainlabel)
    
    train_dataset = Dataset(train_encodings)
    train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
    eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

    # Load or initialize the model
    if count_epoch == 0:
        config = BertConfig.from_pretrained('bert-base-chinese', num_labels=2)
        model = BertForSequenceClassification.from_pretrained("bert-base-chinese", config=config)
    
    # Initialize optimizer and scheduler
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    max_train_steps = num_train_epochs * num_update_steps_per_epoch
    lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=max_train_steps,
    )
    
    accelerator = Accelerator()
    model, optimizer, train_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader
    )
    total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )
    logger.info(accelerator.state)
    logger.info("***** Running training *****")
    logger.info(f"  Num examples = {len(train_dataset)}")
    logger.info(f"  Num Epochs = {num_train_epochs}")
    logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
    logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
    logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
    logger.info(f"  Total optimization steps = {max_train_steps}")

    completed_steps = 0
    for epoch in trange(num_train_epochs, desc="Epoch"):
        model.train()
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            outputs = model(**batch)
            loss = outputs.loss
            accelerator.backward(loss)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            completed_steps += 1
            if step % 200 == 0:
                print({'epoch': epoch, 'step': step, 'loss': loss.item()})
            if completed_steps >= max_train_steps:
                break

        # Evaluation
        logger.info("***** Running eval *****")
        model.eval()
        metric = load_metric("accuracy")
        for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
            with torch.no_grad():
                inputs = {k: v.to(device) for k, v in batch.items()}  
                outputs = model(**inputs)
                predictions = outputs.logits.argmax(dim=-1)
                metric.add_batch(
                    predictions=accelerator.gather(predictions),
                    references=accelerator.gather(batch["labels"]),
                )

        eval_metric = metric.compute()
        logger.info(f"epoch {epoch}: {eval_metric}")
        print("eval accuracy: ", eval_metric['accuracy'])
        all_epoch.append([epoch, eval_metric['accuracy']])
        if eval_metric['accuracy'] > best_epoch['acc']:
            best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(count_epoch), save_function=accelerator.save)

    cnt = 0
    correct_arr = []
    errorcnt = 0
    for i in range(len(train_data)):
        cnt += 1
        sen1 = ori_train_sen1[i]
        sen2 = ori_train_sen2[i]
        predict = eval_model(model, sen1, sen2)  # Need to define eval_model() correctly

        if predict.item() != ori_trainlabel[i]:
            correct_arr.append(-1)
            errorcnt += 1
        else:
            correct_arr.append(1)

    accuracy = (cnt - errorcnt)/cnt
    error_rate = errorcnt / cnt
    alpha = 0.5 * (np.log((1 - error_rate) / error_rate))
    alpha_list.append(alpha)
    #weight *= np.exp(-alpha * trainlabel * correct_arr)
    for i in range(len(weight)):           
        weight[i] *= np.exp(-alpha * correct_arr[i])
    weightdiv = np.sum(weight)  # Normalize the weights
    for i in range(len(weight)):           
        weight[i] /= weightdiv

    print("accuracy: ",accuracy)
    print("alpha: ",alpha)

epoch:  0


06/24/2023 15:48:41 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no

06/24/2023 15:48:41 - INFO - __main__ - ***** Running training *****
06/24/2023 15:48:41 - INFO - __main__ -   Num examples = 5334
06/24/2023 15:48:41 - INFO - __main__ -   Num Epochs = 2
06/24/2023 15:48:41 - INFO - __main__ -   Instantaneous batch size per device = 4
06/24/2023 15:48:41 - INFO - __main__ -   Total train batch size (w. parallel, distributed & accumulation) = 4
06/24/2023 15:48:41 - INFO - __main__ -   Gradient Accumulation steps = 1
06/24/2023 15:48:41 - INFO - __main__ -   Total optimization steps = 2668
Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

{'epoch': 0, 'step': 0, 'loss': 0.4606957733631134}


{'epoch': 0, 'step': 200, 'loss': 0.5990460515022278}


{'epoch': 0, 'step': 400, 'loss': 0.5755136013031006}


...

{'epoch': 1, 'step': 1200, 'loss': 0.0013513725716620684}


Iteration: 100%|█████████▉| 1333/1334 [05:03<00:00,  4.39it/s]
06/24/2023 16:52:14 - INFO - __main__ - ***** Running eval *****
eval Iteration: 100%|██████████| 183/183 [00:14<00:00, 12.55it/s]
06/24/2023 16:52:31 - INFO - __main__ - epoch 1: {'accuracy': 0.6643835616438356}


eval accuracy:  0.6643835616438356


Epoch: 100%|██████████| 2/2 [10:44<00:00, 322.45s/it]


accuracy:  0.9977502812148481
alpha:  3.047348900874817


In [18]:
print(alpha_list)

[0.6163140300431561, 1.219545514618319, 2.049751822516064, 2.903131924781592, 3.047348900874817]


In [ ]:
# alpha_list = [] #投票決定時需要alpha
# for count_epoch in range(total_epoch):
#     train_sen1,train_sen2,trainlabel=read_data(train_data)

#     # tokenize
#     train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)



#     # add label
#     add_targets(train_encodings,trainlabel)


#     # dataload
#     train_dataset = Dataset(train_encodings)




#     from transformers import BertConfig, BertForSequenceClassification
#     if count_epoch==0:
#         config = BertConfig.from_pretrained('bert-base-chinese', num_labels=2)  #num_labels 設定類別數
#         model = BertForSequenceClassification.from_pretrained("bert-base-chinese",config=config)
#     #else:
#     #    config = BertConfig.from_pretrained(f"./epoch_{count_epoch-1}/config.json")
#     #    model = BertForSequenceClassification.from_pretrained(f"./epoch_{count_epoch-1}/pytorch_model.bin", config = config).to(device)

#     import logging
#     import datasets
#     import numpy as np
#     from datasets import load_dataset, load_metric
#     from torch.utils.data import DataLoader
#     from tqdm.auto import tqdm, trange
#     import math

#     import transformers
#     from accelerate import Accelerator
#     from transformers import (
#         AdamW,
#         AutoConfig,
#         default_data_collator,
#         get_scheduler
#     )

#     train_batch_size = 6      # 設定 training batch size
#     eval_batch_size = 6      # 設定 eval batch size
#     num_train_epochs = 1


#     data_collator = default_data_collator
#     train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
#     eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)


#     learning_rate=3e-5          # 設定 learning_rate
#     gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

#     no_decay = ["bias", "LayerNorm.weight"]
#     optimizer_grouped_parameters = [
#         {
#             "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
#             "weight_decay": 0.0,
#         },
#         {
#             "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
#             "weight_decay": 0.0,
#         },
#     ]
#     optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

#     # Scheduler and math around the number of training steps.
#     num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
#     max_train_steps = num_train_epochs * num_update_steps_per_epoch
#     # print('max_train_steps', max_train_steps)

#     # scheduler
#     lr_scheduler = get_scheduler(
#         name="linear",
#         optimizer=optimizer,
#         num_warmup_steps=0,
#         num_training_steps=max_train_steps,
#     )


#     accelerator = Accelerator()

#     # Prepare everything with our `accelerator`.
#     model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
#         model, optimizer, train_dataloader, eval_dataloader
#     )

#     eval_dataloader
#     metric = load_metric("accuracy")


#     #training

#     logger = logging.getLogger(__name__)
#     logging.basicConfig(
#         format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
#         datefmt="%m/%d/%Y %H:%M:%S",
#         level=logging.INFO,
#     )
#     logger.info(accelerator.state)
#     output_dir = '.'  # your folder


#     total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

#     logger.info("***** Running training *****")
#     logger.info(f"  Num examples = {len(train_dataset)}")
#     logger.info(f"  Num Epochs = {num_train_epochs}")
#     logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
#     logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
#     logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
#     logger.info(f"  Total optimization steps = {max_train_steps}")


#     completed_steps = 0


#     for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式
#         model.train()
#         for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
#             outputs = model(**batch)
#             #loss = outputs
#             loss = outputs.loss
#             #loss = loss / gradient_accumulation_steps
#             accelerator.backward(loss)
#             #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: 把if刪掉了
#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad()
#             completed_steps += 1

#             if step % 200 == 0:
#                 print({'epoch': epoch, 'step': step, 'loss': loss.item()})

#             if completed_steps >= max_train_steps:
#                 break

#         logger.info("***** Running eval *****")
#         model.eval()
#         for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
#             outputs = model(**batch)
#             predictions = outputs.logits.argmax(dim=-1)
#             metric.add_batch(
#                 predictions=accelerator.gather(predictions),
#                 references=accelerator.gather(batch["labels"]),
#             )

#         eval_metric = metric.compute()
#         logger.info(f"epoch {epoch}: {eval_metric}")
#         print(eval_metric['accuracy'])
#         all_epoch.append([epoch,eval_metric['accuracy']])
#         if eval_metric['accuracy'] > best_epoch['acc']:
#             best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

#         if output_dir is not None:
#             accelerator.wait_for_everyone()
#             unwrapped_model = accelerator.unwrap_model(model)
#             unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(count_epoch)+'_false>正', save_function=accelerator.save)

#     #訓練完了，要來選擇下次的資料集
#     #因為是計算所有資料所以data使用original data
#     cnt = 0
#     correct_arr = []
#     errorcnt = 0
#     for i in range(len(train_data)):
#         cnt += 1
#         sen1 = ori_train_sen1[i]
#         sen2 = ori_train_sen2[i]
#         predict = eval_model(model, sen1, sen2)  # Need to define eval_model() correctly

#         if predict.item() != ori_trainlabel[i]:
#             correct_arr.append(0)
#             errorcnt += 1
#         else:
#             correct_arr.append(1)

#     error_rate = errorcnt / cnt
#     alpha = 0.5 * (np.log((1 - error_rate) / error_rate))
#     for i in range(len(correct_arr)):
#         if correct_arr[i] == 0:
#             weight[i] *= math.e ** (alpha) 
#         else:
#             weight[i] *= math.e ** (-alpha)

#     weight /= np.sum(weight)  # Normalize the weights
#     alpha_list.append(alpha)
#     print(error_rate, alpha, count_epoch)

真正開始訓練

=========================預測開始=============================

In [30]:
print(np.sum(alpha_list))

9.836092192833949


In [32]:
import numpy as np

def adaboost_voting(model_amount, models, alpha, sen1, sen2):
    votes = 0
    for i in range(model_amount):
        model = models[i]
        model_vote = eval_model(model, sen1, sen2).item()
        votes += model_vote * alpha[i]
    votes /= np.sum(alpha) #調整在0~1之間
    
    #final_prediction = 1 if votes >0.5 else 0 #靠近0就是預測0，靠近1就是預測1
    final_prediction = 1 if votes > 0.5 else 0 #靠近0就是預測0，靠近1就是預測1
    return final_prediction

In [20]:
from transformers import BertConfig, BertForSequenceClassification
models = [] #存多個模型
model_amount = 5 #可以透過這個參數決定要用幾個model預測
for i in range(model_amount):
    config = BertConfig.from_pretrained(f'./epoch_{i}/config.json')
    model = BertForSequenceClassification.from_pretrained(f'./epoch_{i}/pytorch_model.bin', config=config)
    models.append(model)

1.boosting的成效

In [33]:
import numpy as np
cnt=0
correct_arr=[]
errorcnt=0
for i in tqdm(range(len(eval_data))):
    cnt+=1
    sen1=eval_sen1[i]
    sen2=eval_sen2[i]
    predict=adaboost_voting(model_amount, models,alpha_list,sen1,sen2)

    if predict!=evallabel[i]:
        correct_arr.append(0)
        errorcnt+=1
    else:
        correct_arr.append(1)


accuracy=(cnt-errorcnt)/cnt
error_rate = errorcnt/cnt
alpha=0.5*(np.log((1-error_rate)/error_rate))

print(f'cnt = {cnt},errorcnt = {errorcnt}')
print("accuracy: "+str(accuracy))
print("alpha: "+str(alpha))

100%|██████████| 730/730 [01:36<00:00,  7.58it/s]

cnt = 730,errorcnt = 231
accuracy: 0.6835616438356165
alpha: 0.3850941926148626


In [35]:
print(f'cnt = {cnt},errorcnt = {errorcnt}')
print("accuracy: "+str(accuracy))
print("alpha: "+str(alpha))

cnt = 730,errorcnt = 231
accuracy: 0.6835616438356165
alpha: 0.3850941926148626


2.最後一個模型的成效

In [23]:
import numpy as np
model_i=0
for model in models:
    cnt=0
    correct_arr=[]
    errorcnt=0
    for i in tqdm(range(len(eval_data))):
        cnt+=1
        sen1=eval_sen1[i]
        sen2=eval_sen2[i]
        predict=eval_model(model,sen1,sen2)
        if predict.item()!=evallabel[i]:
            correct_arr.append(0)
            errorcnt+=1
        else:
            correct_arr.append(1)
    accuracy=(cnt-errorcnt)/cnt
    error_rate = errorcnt/cnt
    alpha=0.5*(np.log((1-error_rate)/error_rate))
    print(f"model {model_i}: ")
    print(f'cnt = {cnt},errorcnt = {errorcnt}')
    print("accuracy: "+str(accuracy))
    print("alpha: "+str(alpha))
    model_i+=1

100%|██████████| 730/730 [00:19<00:00, 38.11it/s]


model 0: 
cnt = 730,errorcnt = 239
accuracy: 0.6726027397260274
alpha: 0.359990287931505


100%|██████████| 730/730 [00:19<00:00, 38.02it/s]


model 1: 
cnt = 730,errorcnt = 223
accuracy: 0.6945205479452055
alpha: 0.41066961606553215


100%|██████████| 730/730 [00:19<00:00, 37.89it/s]


model 2: 
cnt = 730,errorcnt = 234
accuracy: 0.6794520547945205
alpha: 0.375627405683613


100%|██████████| 730/730 [00:19<00:00, 37.70it/s]


model 3: 
cnt = 730,errorcnt = 231
accuracy: 0.6835616438356165
alpha: 0.3850941926148626


100%|██████████| 730/730 [00:19<00:00, 37.81it/s]

model 4: 
cnt = 730,errorcnt = 245
accuracy: 0.6643835616438356
alpha: 0.3414453401963781


=========================預測結束=============================

## 最好的某次訓練成果

In [ ]:
print(best_epoch)

In [ ]:
print(all_epoch)

# 訓練成果驗證

### 可以拿來玩的地方

結果存進google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd content/gdrive/MyDrive/Colab Notebooks

In [ ]:
cd NTCIR

In [ ]:
config = BertConfig.from_pretrained("/content/epoch_4/config.json")
model = BertForSequenceClassification.from_pretrained("/content/epoch_4/pytorch_model.bin", config = config)
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained("./"+output_dir + '/' + 'epoch_' + str(4), save_function=accelerator.save)